In [ ]:
%load_ext autoreload
%autoreload 2
from itertools  import pairwise
from threader   import Threader
from geometry   import GSegment, GPoint, list2gsegments
from tlayer     import TLayer
from gcode_file import GcodeFile
from pathlib    import Path
from util       import deep_update

from Geometry3D import HalfLine, Segment, Point
from Geometry3D.calc import intersection

import logger, plotting, plot_styles
import plot_helpers, util
import sys
import plotly.graph_objects as go

from plot_helpers import quickplot

#This module has settings specific to our current printer setup
try:
    import ender3v2 as ender3
except ImportError:
    import ender3
    
from manualprinter import Manualprinter

In [ ]:
test_root = Path('./example/basic test/')
gcode_filename = test_root.joinpath('basic test-cura.gcode')
thread_geom = [
    (47, 70, .4),
    (65, 80, .4),
    (65, 93.02, .8),
    (60.042, 99.364, .8),
    (69.113, 106.435, .8),
    (69.113, 111.435, .8),
]

In [ ]:
logger.restart_logging()
gcode = GcodeFile(gcode_filename, layer_class=TLayer)
threader = Threader(gcode, 'manualprinter.yaml', thread_geom)
threader.route_model()

In [ ]:
threader.save(test_root.joinpath('basic test-routed.gcode'))

In [ ]:
paper_styles = deep_update(plot_helpers.styles, {
    'future_thread': {'line': dict(color='#b28600',    width=3,  dash='dash')},
    'gc_segs':       {'line': dict(width=2)},
    #'old_segs':      {'line': dict(width=2)},
    'to_print':      {'line': dict(color='#1192e8', dash='dot'), 'opacity': 1},
    'thread_ring':   {'line': dict(color='#b28600', width=3, dash='dash')},
    'printed_thread':{'line': dict(color='#b28600', width=3, dash=None)},
    'anchor':        {'marker': dict(symbol='circle', size=6)},
    'original_thread': {'line': dict(color='gray', width=2, dash='dot')},
    'original_anchor': {'marker': dict(color='gray', symbol='circle', size=6)},
})

In [ ]:
# test_layer = 3
figs = plotting.plot_steps(threader.layer_steps[test_layer], template='plotly_white',
    styles=paper_styles,
    zoom_box=((60.2, 90.2), (69.8, 109.8)),
    show_args={
        'xaxis':dict(showgrid=False, showticklabels=False),
        'yaxis':dict(showgrid=False, showticklabels=False)
    }, show=False
                          )#, prev_layer=gcode.layers[test_layer-1], prev_layer_only_outline=False, preview_layer=False)
#plotting.plot_test(threader.layer_steps[1])

In [ ]:
fig = go.Figure()
plot_helpers.plot_segments(fig, gcode.layers[3].geometry.segments, style=paper_styles['gc_segs'])
plot_helpers.plot_segments(fig, [GSegment(a,b) for (a,b) in pairwise(thread_geom) if a != b], style=paper_styles['original_thread'], mode='markers+lines', **paper_styles['original_anchor'])
plot_helpers.plot_segments(fig, [GSegment(a,b) for (a,b) in pairwise([step.thread_path.point for step in threader.layer_steps[3].steps]) if a != b], style=paper_styles['future_thread'], mode='markers+lines', **paper_styles['anchor'])
plot_helpers.show_fig(fig, template='plotly_white',
    zoom_box=((60.2, 90.2), (69.8, 109.8)),
    xaxis=dict(showgrid=False, showticklabels=False),
    yaxis=dict(showgrid=False, showticklabels=False),
)
fig.write_image('route-preview.pdf')

In [ ]:
for i,fig in enumerate(figs):
    fig.write_image(f'route{i:02}.pdf')

In [ ]:
%pip install kaleido

In [ ]:
test_layer = 3
plotting.plot_steps(threader.layer_steps[test_layer], prev_layer=gcode.layers[test_layer-1], prev_layer_only_outline=False)